In [1]:
# Read the KML file as bytes
file_path = "./Building_Footprint.kml"

with open(file_path, "rb") as file:
    kml_data = file.read()

# Decode and remove XML declaration (to inspect the raw XML)
kml_str = kml_data.decode("utf-8").split("?>", 1)[-1].strip()

# Print the raw XML content for inspection
print("Raw KML Content:\n")
print(kml_str[:1000])  # Print first 1000 characters for a quick look

# Optionally, write the output to a file to inspect the full XML content if needed
with open("output_kml_structure.xml", "w", encoding="utf-8") as output_file:
    output_file.write(kml_str)


Raw KML Content:

<kml xmlns="http://www.opengis.net/kml/2.2">
<Document id="root_doc">
<Schema name="Challenge_footprint" id="Challenge_footprint">
	<SimpleField name="timestamp" type="string"></SimpleField>
	<SimpleField name="begin" type="string"></SimpleField>
	<SimpleField name="end" type="string"></SimpleField>
	<SimpleField name="altitudeMode" type="string"></SimpleField>
	<SimpleField name="tessellate" type="int"></SimpleField>
	<SimpleField name="extrude" type="int"></SimpleField>
	<SimpleField name="visibility" type="int"></SimpleField>
	<SimpleField name="drawOrder" type="int"></SimpleField>
	<SimpleField name="icon" type="string"></SimpleField>
	<SimpleField name="id" type="string"></SimpleField>
	<SimpleField name="fid" type="int"></SimpleField>
	<SimpleField name="layer" type="string"></SimpleField>
	<SimpleField name="path" type="string"></SimpleField>
</Schema>
<Folder><name>Challenge_footprint</name>
  <Placemark>
	<Style><LineStyle><color>ff0000ff</color></LineStyle><

In [2]:
from fastkml import kml
from lxml import etree

# Path to the KML file
file_path = "./Building_Footprint.kml"

# Read the KML file as bytes
with open(file_path, "rb") as file:
    kml_data = file.read()

# Decode and remove XML declaration (to inspect the raw XML)
kml_str = kml_data.decode("utf-8").split("?>", 1)[-1].strip()

# Parse KML string using lxml to handle namespaces properly
root = etree.fromstring(kml_str)

# Use the correct namespace
namespace = {'kml': 'http://www.opengis.net/kml/2.2'}

# Extract the Folder and Placemark elements
folders = root.findall('.//kml:Folder', namespace)

# Check if we found folders and placemarks
if not folders:
    print("⚠️ No folders found in the KML file.")
else:
    print("Folders Found:")
    for folder in folders:
        # Safely extract the folder name
        folder_name = folder.find('kml:name', namespace)
        if folder_name is not None:
            print(f"  Folder Name: {folder_name.text}")
        else:
            print("  Folder Name: (No name provided)")

        # Extract placemarks within this folder
        for placemark in folder.findall('kml:Placemark', namespace):
            # Safely extract the placemark name
            placemark_name = placemark.find('kml:name', namespace)
            if placemark_name is not None:
                placemark_label = placemark_name.text
            else:
                placemark_label = "(No name provided)"

            # Print placemark name
            print(f"    Placemark Name: {placemark_label}")
            
            # Extract coordinates or other geometry
            coordinates = placemark.findall('.//kml:coordinates', namespace)
            if coordinates:
                print("      Coordinates:")
                for coord in coordinates:
                    # Format the coordinates
                    coord_list = coord.text.strip().split()
                    for coordinate in coord_list:
                        lat, lon = coordinate.split(',')
                        print(f"        Latitude: {lat}, Longitude: {lon}")
            else:
                print("      Coordinates: (No coordinates found)")


Folders Found:
  Folder Name: Challenge_footprint
    Placemark Name: (No name provided)
      Coordinates:
        Latitude: -73.919029, Longitude: 40.848199
        Latitude: -73.919329, Longitude: 40.847953
        Latitude: -73.919371, Longitude: 40.847982
        Latitude: -73.919453, Longitude: 40.847915
        Latitude: -73.919534, Longitude: 40.847972
        Latitude: -73.919152, Longitude: 40.848285
        Latitude: -73.919029, Longitude: 40.848199
    Placemark Name: (No name provided)
      Coordinates:
        Latitude: -73.921952, Longitude: 40.849629
        Latitude: -73.921911, Longitude: 40.849679
        Latitude: -73.921764, Longitude: 40.84961
        Latitude: -73.921805, Longitude: 40.84956
        Latitude: -73.921952, Longitude: 40.849629
    Placemark Name: (No name provided)
      Coordinates:
        Latitude: -73.920502, Longitude: 40.850113
        Latitude: -73.920453, Longitude: 40.850167
        Latitude: -73.920262, Longitude: 40.850069
        Latit

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [6]:
conda install -c conda-forge geopandas


Retrieving notices: ...working... done

NoSpaceLeftError: No space left on devices.


Note: you may need to restart the kernel to use updated packages.


In [5]:
from lxml import etree
import geopandas as gpd
from shapely.geometry import Polygon

# Function to extract coordinates from KML format
def parse_kml(kml_file):
    tree = etree.parse(kml_file)
    root = tree.getroot()
    
    # Find all coordinates inside <coordinates> tags
    coordinates = []
    for placemark in root.findall('.//Placemark'):
        coord_str = placemark.find('.//coordinates').text.strip()
        coords = [tuple(map(float, coord.split(','))) for coord in coord_str.split()]
        coordinates.append(coords)
    
    return coordinates

# Load KML and extract coordinates from "Building_Footprint" KML file
kml_file = 'Building_Footprint.kml'
coordinates = parse_kml(kml_file)

# Convert coordinates to polygons
polygons = [Polygon(coords) for coords in coordinates]

# Create GeoDataFrame from the polygons
gdf_buildings = gpd.GeoDataFrame(geometry=polygons, crs="EPSG:4326")

# Check the result
print(gdf_buildings.head())


ModuleNotFoundError: No module named 'geopandas'